## 1. Setup and Imports

In [ ]:
# Install dependencies (run once)
# !pip install -r ../requirements.txt

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Set display options
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

print("Setup complete!")

## 2. Image Preprocessing

In [ ]:
from src.preprocessing import ImagePreprocessor

# Initialize preprocessor
preprocessor = ImagePreprocessor(
    target_height=64,
    target_width=256,
    binarization_method='adaptive',
    denoise=True,
    deskew=True
)

print("Preprocessor initialized!")

In [ ]:
# Create a sample image with text
sample_image = np.ones((100, 300), dtype=np.uint8) * 255
cv2.putText(sample_image, "Sample Text", (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.5, 0, 3)

# Add some noise
noise = np.random.randint(0, 30, sample_image.shape, dtype=np.uint8)
noisy_image = cv2.add(sample_image, noise)

# Preprocess
processed_image = preprocessor.preprocess(noisy_image)

# Display results
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].imshow(sample_image, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(noisy_image, cmap='gray')
axes[1].set_title('Noisy Image')
axes[1].axis('off')

axes[2].imshow(processed_image, cmap='gray')
axes[2].set_title('Preprocessed Image')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"Original shape: {sample_image.shape}")
print(f"Processed shape: {processed_image.shape}")

## 3. Script Classification

In [ ]:
from src.script_classifier import ScriptClassifier, SCRIPT_LABELS

print("Supported Scripts:")
for idx, name in SCRIPT_LABELS.items():
    print(f"  {idx}: {name}")

In [ ]:
# Initialize script classifier
classifier = ScriptClassifier(
    model_type='cnn',
    num_classes=3,
    pretrained=True
)

print(f"Classifier initialized on: {classifier.device}")

In [ ]:
# Test prediction on dummy image
dummy_image = np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8)

script, confidence = classifier.predict(dummy_image)

print(f"Predicted Script: {script}")
print(f"Confidence: {confidence:.4f}")

# Note: With untrained model, predictions will be random

## 4. OCR Engine

In [ ]:
# Note: OCR engine requires paddleocr or easyocr to be installed
# Uncomment below to test if dependencies are available

# from src.ocr_engine import OCREngine

# try:
#     ocr_engine = OCREngine(engine='paddleocr', use_gpu=False)
#     print("OCR Engine initialized successfully!")
# except ImportError as e:
#     print(f"OCR engine not available: {e}")
#     print("Install with: pip install paddleocr")

## 5. Evaluation Metrics

In [ ]:
from src.evaluation import (
    character_error_rate,
    word_error_rate,
    OCRMetrics,
    ClassificationMetrics
)

# Test CER and WER
test_cases = [
    ("hello world", "hello world"),  # Perfect match
    ("hallo world", "hello world"),  # 1 substitution
    ("hello", "hello world"),        # Missing word
    ("test", "testing"),              # Partial match
]

print("OCR Evaluation Examples:")
print("=" * 60)

for predicted, ground_truth in test_cases:
    cer = character_error_rate(predicted, ground_truth)
    wer = word_error_rate(predicted, ground_truth)
    
    print(f"Predicted:    '{predicted}'")
    print(f"Ground Truth: '{ground_truth}'")
    print(f"CER: {cer:.4f} | WER: {wer:.4f}")
    print("-" * 40)

In [ ]:
# Test with Indic text
indic_test_cases = [
    ("नमस्ते भारत", "नमस्ते भारत"),  # Hindi - exact match
    ("வணக்கம்", "வணக்கம்"),        # Tamil - exact match
    ("നമസ്കാരം", "നമസ്കാരം"),      # Malayalam - exact match
]

print("Indic Script Evaluation:")
print("=" * 60)

for predicted, ground_truth in indic_test_cases:
    cer = character_error_rate(predicted, ground_truth)
    
    print(f"Text: '{predicted}'")
    print(f"CER: {cer:.4f}")
    print("-" * 40)

In [ ]:
# Comprehensive evaluation example
metrics = OCRMetrics()

# Add sample results
sample_results = [
    ("hello world", "hello world"),
    ("hallo wrld", "hello world"),
    ("namaste", "namaste"),
    ("test sample", "testing samples"),
    ("OCR result", "OCR result"),
]

for i, (pred, gt) in enumerate(sample_results):
    metrics.add_result(pred, gt, image_id=f"sample_{i}")

# Print summary
metrics.print_summary()

## 6. Dataset Utilities

In [ ]:
from src.dataset import DatasetManager, SyntheticDataGenerator

# Initialize dataset manager
dm = DatasetManager(base_dir='../data')

# Get dataset statistics
stats = dm.get_dataset_stats()
print("Dataset Statistics:")
print(f"  Total images: {stats['total_images']}")
print(f"  Scripts: {stats['scripts']}")

## 7. Utility Functions

In [ ]:
from src.utils import (
    is_indic_text,
    detect_script_from_text,
    get_unicode_range
)

# Test text samples
samples = [
    "Hello World",           # English
    "नमस्ते",                  # Hindi/Devanagari
    "வணக்கம்",               # Tamil
    "നമസ്കാരം",              # Malayalam
    "Mixed नमस्ते text",      # Mixed
]

print("Script Detection from Text:")
print("=" * 50)

for text in samples:
    is_indic = is_indic_text(text)
    detected = detect_script_from_text(text)
    
    print(f"Text: '{text}'")
    print(f"  Is Indic: {is_indic}")
    print(f"  Detected Script: {detected}")
    print("-" * 40)

## 8. Complete Pipeline Demo

In [ ]:
# This demonstrates the complete pipeline
# Note: Actual OCR requires trained models and OCR engine

def demo_pipeline(image_path=None):
    """
    Demonstrate the complete OCR pipeline.
    """
    print("Indic-OCR Pipeline Demo")
    print("=" * 50)
    
    # Step 1: Load/Create image
    print("\n[Step 1] Loading image...")
    if image_path:
        image = cv2.imread(image_path)
    else:
        # Create demo image
        image = np.ones((100, 400), dtype=np.uint8) * 255
        cv2.putText(image, "Demo OCR Text", (20, 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.5, 0, 2)
    print(f"   Image shape: {image.shape}")
    
    # Step 2: Preprocess
    print("\n[Step 2] Preprocessing...")
    preprocessor = ImagePreprocessor()
    processed = preprocessor.preprocess(image)
    print(f"   Preprocessed shape: {processed.shape}")
    
    # Step 3: Script Classification
    print("\n[Step 3] Detecting script...")
    classifier = ScriptClassifier(model_type='cnn')
    script, confidence = classifier.predict(processed)
    print(f"   Detected: {script} (confidence: {confidence:.2%})")
    
    # Step 4: OCR (simulated)
    print("\n[Step 4] Performing OCR...")
    print("   (OCR engine would extract text here)")
    simulated_text = "Demo OCR Text"  # Simulated result
    print(f"   Extracted text: '{simulated_text}'")
    
    print("\n" + "=" * 50)
    print("Pipeline demo complete!")
    
    return {
        'script': script,
        'confidence': confidence,
        'text': simulated_text
    }

# Run demo
result = demo_pipeline()

## 9. Next Steps

1. **Collect Dataset**: Gather images of text in Devanagari, Malayalam, and Tamil scripts
2. **Train Script Classifier**: Run `python train.py --task script_classifier`
3. **Test OCR Engine**: Install PaddleOCR and test with real images
4. **Evaluate Performance**: Use the evaluation metrics on your test set
5. **Deploy Web App**: Run `streamlit run app/streamlit_app.py`

In [ ]:
print("\n" + "=" * 60)
print("Indic-OCR Notebook Complete!")
print("=" * 60)
print("\nFor more information, see README.md")
print("Author: Dhananjayan H")
print("Roll No: AA.SC.P2MCA24070151")